In [1]:
from ultralytics import YOLO
import torch
import os
import fnmatch
import numpy as np
from sklearn import metrics

In [2]:
def data_loder(path):
    img_paths = []
    labels = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, "*.jpg"):
                labels.append(0 if '0_real' in root else 1)
                img_paths.append(os.path.join(root, name))
    return (img_paths, labels)

In [3]:
def evaluate(model, test_loader, batch_size):
    # Validation phase
    model.eval()
    
    y_true = np.array(test_loader[1])
    
    img_paths = test_loader[0]
    y_pred = []

    # Process images in batches
    for i in range(0, len(img_paths), batch_size):
        j = min(i + batch_size, len(img_paths))
        batch_paths = img_paths[i:j]
        results = model.predict(batch_paths, verbose=False)  # Predict batch
        y_pred.extend([result.probs.top1 for result in results])  # Collect predictions

    y_pred = np.array(y_pred)
    accuracy = metrics.accuracy_score(y_true, y_pred)
    return accuracy

In [4]:
def load_model(model_path):
    model = YOLO(model_path, task='classify', verbose=False)
    model.to(DEVICE)
    print(f'Model loaded from {model_path}')
    return model

In [5]:
if __name__ == "__main__":
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   
    #path to the model weights
    model_weight_filename = r'epoch3.pt'
    
    model = load_model(model_weight_filename)
    
    
    # Load the data
    test_dataset_path = r'..\AIGC-Detection-Dataset\test'
    batch_size = 512
    
    test_loader = data_loder(test_dataset_path)
    
    accuracy = evaluate(model, test_loader, batch_size)
    print(f"Accuracy: {accuracy}")

Model loaded from epoch3.pt
Accuracy: 0.6896551724137931
